In [3]:
import cv2
import numpy as np
import sys
import os
import time
import argparse
import json
import requests
import base64
import io
import matplotlib.pyplot as plt
import moviepy.editor as mpy

## Thresholding


In [45]:

def abs_sobel_thresh(image, orient='x', sobel_kernel=3, thresh_min=20, thresh_max=110):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    if orient == 'x':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    elif orient == 'y':
        sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    abs_sobel = np.absolute(sobel)
    scaled_sobel = np.uint8(255 * abs_sobel / np.max(abs_sobel))
    binary_output = np.zeros_like(scaled_sobel)
    binary_output[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    return binary_output

def mag_thresh(image, sobel_kernel=3, mag_thresh=(30, 100)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    grad_mag = np.sqrt(sobel_x**2 + sobel_y**2)
    scaled_mag = np.uint8(255 * grad_mag / np.max(grad_mag))
    binary_output = np.zeros_like(scaled_mag)
    binary_output[(scaled_mag >= mag_thresh[0]) & (scaled_mag <= mag_thresh[1])] = 1
    return binary_output

def dir_threshold(image, sobel_kernel=3, thresh=(0.7, 1.3)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    sobel_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobel_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    grad_dir = np.arctan2(np.absolute(sobel_y), np.absolute(sobel_x))
    binary_output = np.zeros_like(grad_dir)
    binary_output[(grad_dir >= thresh[0]) & (grad_dir <= thresh[1])] = 1
    return binary_output

def lab_threshold(image, thresh_l=(200, 255), thresh_a=(-128, 128), thresh_b=(-128, 128)):
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
    L, a, b = cv2.split(lab)
    
    l_binary = np.zeros_like(L)
    l_binary[(L > thresh_l[0]) & (L <= thresh_l[1])] = 1
    
    a_binary = np.zeros_like(a)
    a_binary[(a > thresh_a[0]) & (a <= thresh_a[1])] = 1
    
    b_binary = np.zeros_like(b)
    b_binary[(b > thresh_b[0]) & (b <= thresh_b[1])] = 1
    
    return l_binary, a_binary, b_binary


def combined_gradient(image):
    ksize = 7  # Sobel kernel size

    # Reduce the number of transformations
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh_min=20, thresh_max=110)
    grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh_min=20, thresh_max=110)
    mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(90, 200))
    
    # Combine grad_x, grad_y and mag_binary (dir_binary removed for speed)
    combined = np.zeros_like(mag_binary)
    combined[(gradx == 1) | (grady == 1) | (mag_binary == 1)] = 1
    
    return combined

### Select Points for ROI

In [5]:
# Lista para almacenar los puntos seleccionados
selected_points = []

# Función para manejar la selección de puntos en la imagen
def select_points(event, x, y, flags, param):
    global selected_points
    if event == cv2.EVENT_LBUTTONDOWN:
        # Al hacer clic izquierdo, agregamos el punto seleccionado a la lista
        if len(selected_points) < 4:  # Solo permitir 4 puntos
            selected_points.append((x, y))
            print(f"Point {len(selected_points)} selected: ({x}, {y})")
        if len(selected_points) == 4:
            print("4 points selected. Press 'q' to continue.")

# Función para seleccionar los puntos en la imagen
def select_points_on_image(image):
    global selected_points
    selected_points = []  # Limpiar puntos seleccionados antes de comenzar

    # Crear una ventana para mostrar la imagen
    cv2.imshow("Select 4 Points", image)
    cv2.setMouseCallback("Select 4 Points", select_points)

    # Esperar a que el usuario seleccione los 4 puntos
    while len(selected_points) < 4:
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):  # Permitir salir si presionas 'q'
            break

    cv2.destroyAllWindows()

    # Devolver los puntos seleccionados
    return selected_points


# Select the ROI on the diferent images AND SELECT the ROI as a mean of the 4 points selected
# THE ORDER OF SELECTION IS bottom left, top left, bottom right, top right
def select_points_on_images(images):
    sample_images_idx = np.random.choice(len(images), 6, replace=False)
    roi_points = []
    for idx in sample_images_idx:
        image = images[idx]
        selected_points = select_points_on_image(image)
        roi_points.append(selected_points)
        
    # do the mean for the 4 points selected for each image
    roi_points = np.mean(roi_points, axis=0)
    print(roi_points)
    return roi_points


# Función para dibujar el ROI en la imagen
def draw_roi(image, roi_points):
    if len(roi_points) == 4:
        # Los puntos se asumen como el orden: p1, p2, p4, p3 (base inferior a base superior)
        pts = np.array(roi_points, dtype=int)

        # Dibujar el ROI (polígono) en la imagen
        image_with_roi = image.copy()
        cv2.polylines(image_with_roi, [pts], isClosed=True, color=(0, 255, 0), thickness=2)

        # Mostrar la imagen con el ROI seleccionado
        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(image_with_roi, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.show()

    else:
        print("No se seleccionaron suficientes puntos.")

### Perspective Transform

In [25]:
def apply_perspective_transform(image, roi_points):
    # Definir los 4 puntos de destino para la transformación
    height, width = image.shape[:2]
    dst_points = np.array([[0, height], [0, 0], [width, 0], [width, height]], dtype=np.float32)
    src_points = np.array(roi_points, dtype=np.float32)

    # Calcular la matriz de transformación de perspectiva
    M = cv2.getPerspectiveTransform(src_points, dst_points)

    # Aplicar la transformación de perspectiva
    warped_image = cv2.warpPerspective(image, M, (width, height))

    return warped_image, dst_points, src_points


def plot_ROI_and_perspective_transform(image, roi_points):
    if len(roi_points) == 4:
        warped_image, dst, src = apply_perspective_transform(image, roi_points)

        # Visualizar la imagen original con los puntos seleccionados
        plt.figure(figsize=(20, 10))
        plt.subplot(1, 2, 1)
        plt.title("Original Image with Selected ROI")
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        poly = plt.Polygon(roi_points, closed=True, fill=False, color='#FF0000')
        plt.gca().add_patch(poly)

        # Visualizar la imagen transformada
        plt.subplot(1, 2, 2)
        plt.title("Warped Image")
        plt.imshow(cv2.cvtColor(warped_image, cv2.COLOR_BGR2RGB))
        plt.show()

    else:
        print("No se seleccionaron suficientes puntos.")

    binary_warped = combined_gradient(warped_image)
 
    plt.figure(figsize=(10, 10))
    plt.imshow(binary_warped, cmap='gray')
    plt.title("Binary Warped Image")
    plt.axis('off')
    plt.show()

### Histogram

In [48]:
def calculate_histogram(binary_warped):
    """Calcula el histograma de la mitad inferior de una imagen binaria."""
    return np.sum(binary_warped[binary_warped.shape[0]//2:, :], axis=0)

def find_lane_base(histogram):
    """Encuentra las bases de las líneas izquierda y derecha usando el histograma."""
    midpoint = int(histogram.shape[0] // 2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint # We add the midpoint to get the offset
    return leftx_base, rightx_base



### Sliding Window

In [24]:
def perform_sliding_window(binary_image, base_left, base_right, num_windows=15, window_margin= 150, min_pixels=300):
    """
    Implementa el algoritmo del sliding window para detectar los puntos de carril.
    
    Parameters:
        binary_image: Imagen binaria transformada (con perspectiva corregida).
        base_left, base_right: Coordenadas base iniciales de los carriles izquierdo y derecho.
        num_windows: Número de ventanas a utilizar.
        window_margin: Ancho de cada ventana en píxeles.
        min_pixels: Umbral mínimo de píxeles para recenter una ventana.
    
    Returns:
        left_curve, right_curve: Polinomios ajustados para los carriles izquierdo y derecho.
        left_pixels, right_pixels: Índices de los píxeles pertenecientes a cada carril.
        detected_windows: Lista de coordenadas centrales de las ventanas detectadas.
    """
    # Altura de cada ventana en píxeles
    height_window = binary_image.shape[0] // num_windows

    # Obtener las posiciones de los píxeles activados
    pixels_y, pixels_x = binary_image.nonzero()

    # Posición actual de las ventanas
    current_left = base_left
    current_right = base_right

    # Listas para guardar los píxeles del carril y los centros de las ventanas
    left_indices = []
    right_indices = []
    detected_windows = []

    for win in range(num_windows):
        # Límites de las ventanas
        y_top = binary_image.shape[0] - (win + 1) * height_window
        y_bottom = binary_image.shape[0] - win * height_window
        x_left_low = current_left - window_margin
        x_left_high = current_left + window_margin
        x_right_low = current_right - window_margin
        x_right_high = current_right + window_margin

        # Agregar las coordenadas centrales de las ventanas
        detected_windows.append(((current_left, y_bottom), (current_right, y_bottom)))

        # Encontrar píxeles dentro de cada ventana
        valid_left = ((pixels_y >= y_top) & (pixels_y < y_bottom) &
                      (pixels_x >= x_left_low) & (pixels_x < x_left_high)).nonzero()[0]
        valid_right = ((pixels_y >= y_top) & (pixels_y < y_bottom) &
                       (pixels_x >= x_right_low) & (pixels_x < x_right_high)).nonzero()[0]

        # Guardar los índices de los píxeles
        left_indices.append(valid_left)
        right_indices.append(valid_right)

        # Actualizar la posición de las ventanas si se detectan suficientes píxeles
        if len(valid_left) > min_pixels:
            current_left = int(np.mean(pixels_x[valid_left]))
        if len(valid_right) > min_pixels:
            current_right = int(np.mean(pixels_x[valid_right]))

    # Concatenar índices de píxeles
    left_indices = np.concatenate(left_indices)
    right_indices = np.concatenate(right_indices)

    # Extraer las coordenadas de los píxeles de cada carril
    x_left = pixels_x[left_indices]
    y_left = pixels_y[left_indices]
    x_right = pixels_x[right_indices]
    y_right = pixels_y[right_indices]

    # Ajustar polinomios a los píxeles detectados
    left_curve = np.polyfit(y_left, x_left, 2) if len(x_left) > 0 else None
    right_curve = np.polyfit(y_right, x_right, 2) if len(x_right) > 0 else None

    return left_curve, right_curve, left_indices, right_indices, detected_windows



def visualize_results(binary_image, left_fit, right_fit, window_coords):
    """
    Dibuja las ventanas del sliding window y los polinomios ajustados.
    
    Parameters:
        binary_image: Imagen binaria de entrada.
        left_fit, right_fit: Coeficientes de los polinomios ajustados.
        window_coords: Coordenadas de las ventanas del sliding window.
    """
    y_vals = np.linspace(0, binary_image.shape[0] - 1, binary_image.shape[0])
    x_left_fit = x_right_fit = None

    if left_fit is not None:
        x_left_fit = left_fit[0] * y_vals ** 2 + left_fit[1] * y_vals + left_fit[2]
    if right_fit is not None:
        x_right_fit = right_fit[0] * y_vals ** 2 + right_fit[1] * y_vals + right_fit[2]

    plt.imshow(binary_image, cmap='gray')
    
    # Dibujar ventanas en funcion del size de la imagen
    for (left, right) in window_coords:
        left_rect = plt.Rectangle((left[0] - 60, left[1] - 25), 120, 50, edgecolor='cyan', fill=False, linewidth=2)
        right_rect = plt.Rectangle((right[0] - 60, right[1] - 25), 120, 50, edgecolor='orange', fill=False, linewidth=2)
        plt.gca().add_patch(left_rect)
        plt.gca().add_patch(right_rect)
    
    # Dibujar polinomios ajustados
    if x_left_fit is not None:
        plt.plot(x_left_fit, y_vals, color='lime', label='Carril Izquierdo')
    if x_right_fit is not None:
        plt.plot(x_right_fit, y_vals, color='magenta', label='Carril Derecho')
    
    plt.title("Sliding Windows y Ajuste de Carriles")
    plt.legend()
    plt.show()



### Downsample Image

In [8]:
def downsample_image(image, target_size = (800,600)):
    """
    Redimensiona la imagen de entrada a un tamaño objetivo.
    
    Parameters:
        image: Imagen de entrada.
        target_size: Tamaño objetivo de la imagen (ancho, alto).
    
    Returns:
        Imagen redimensionada.
    """
    return cv2.resize(image, target_size, interpolation=cv2.INTER_LINEAR)


    

### Curve Information

In [9]:

def calculate_curvature(left_fit, right_fit, y_eval):
    """
    Calcula el radio de curvatura de los carriles izquierdo y derecho.
    
    Parameters:
        left_fit, right_fit: Polinomios ajustados para los carriles izquierdo y derecho.
        y_eval: La posición y (en píxeles) en la que se calcula la curvatura (generalmente en el fondo de la imagen).
        
    Returns:
        left_curvature, right_curvature: Radios de curvatura de los carriles izquierdo y derecho.
    """
    # Coeficientes de los polinomios para el carril izquierdo y derecho
    A_left, B_left, C_left = left_fit
    A_right, B_right, C_right = right_fit
    
    # Calculamos el radio de curvatura usando la fórmula
    left_curvature = ((1 + (2 * A_left * y_eval + B_left) ** 2) ** 1.5) / np.abs(2 * A_left)
    right_curvature = ((1 + (2 * A_right * y_eval + B_right) ** 2) ** 1.5) / np.abs(2 * A_right)
    
    return left_curvature, right_curvature

def detect_car_direction(left_fit, right_fit):
    """
    Detecta si el coche está yendo recto, girando a la izquierda o girando a la derecha.
    
    Parameters:
        left_fit, right_fit: Polinomios ajustados para los carriles izquierdo y derecho.
    
    Returns:
        direction: Dirección del coche ("Recto", "Izquierda", "Derecha").
    """
    # Comprobar si los coeficientes A de los polinomios son cercanos a cero
    A_left, _, _ = left_fit
    A_right, _, _ = right_fit
    
    if np.abs(A_left) < 0.0001 and np.abs(A_right) < 0.0001:
        direction = "Recto"
    elif A_left > 0 and A_right > 0:
        direction = "Derecha"
    elif A_left < 0 and A_right < 0:
        direction = "Izquierda"
    else:
        direction = "Recto"
    
    return direction


def visualize_results_with_curvature(binary_image, left_fit, right_fit, window_coords, y_eval, original_image):
    """
    Dibuja las ventanas del sliding window, los polinomios ajustados y los radios de curvatura,
    junto con la imagen original.
    
    Parameters:
        binary_image: Imagen binaria de entrada.
        left_fit, right_fit: Coeficientes de los polinomios ajustados.
        window_coords: Coordenadas de las ventanas del sliding window.
        y_eval: Posición y (en píxeles) en la que se calcula la curvatura.
        original_image: Imagen original de entrada.
    """
    y_vals = np.linspace(0, binary_image.shape[0] - 1, binary_image.shape[0])
    x_left_fit = x_right_fit = None

    if left_fit is not None:
        x_left_fit = left_fit[0] * y_vals ** 2 + left_fit[1] * y_vals + left_fit[2]
    if right_fit is not None:
        x_right_fit = right_fit[0] * y_vals ** 2 + right_fit[1] * y_vals + right_fit[2]

    # Crear figura con 2 subgráficas (una para la imagen original y otra para la imagen procesada)
    fig, ax = plt.subplots(1, 2, figsize=(14, 7))

    # Mostrar la imagen original en la primera subgráfica
    ax[0].imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    ax[0].set_title("Imagen Original")
    ax[0].axis('off')  # Ocultar ejes

    # Mostrar la imagen binaria procesada en la segunda subgráfica
    ax[1].imshow(binary_image, cmap='gray')
    
    # Dibujar ventanas
    for (left, right) in window_coords:
        left_rect = plt.Rectangle((left[0] - 60, left[1] - 25), 120, 50, edgecolor='cyan', fill=False, linewidth=2)
        right_rect = plt.Rectangle((right[0] - 60, right[1] - 25), 120, 50, edgecolor='orange', fill=False, linewidth=2)
        ax[1].add_patch(left_rect)
        ax[1].add_patch(right_rect)
    
    # Dibujar polinomios ajustados
    if x_left_fit is not None:
        ax[1].plot(x_left_fit, y_vals, color='lime', label='Carril Izquierdo')
    if x_right_fit is not None:
        ax[1].plot(x_right_fit, y_vals, color='magenta', label='Carril Derecho')
    
    # Calcular los radios de curvatura
    left_curvature, right_curvature = calculate_curvature(left_fit, right_fit, y_eval)
    direction = detect_car_direction(left_fit, right_fit)

    # Radio de curvatura global
    global_curvature = (left_curvature + right_curvature) / 2
    curvature_text = f"Curvatura: {global_curvature:.2f} m"
    direction_text = f"Dirección: {direction}"

    ax[1].set_title("Sliding Windows y Ajuste de Carriles")
    ax[1].legend()
    ax[1].text(50, 50, curvature_text, color='orange', fontsize=12, fontweight='bold')
    ax[1].text(50, 100, direction_text, color='orange', fontsize=12, fontweight='bold')

    plt.tight_layout()  # Ajustar el espacio entre subgráficas
    plt.show()

### ROI video output

In [21]:
def execute_ROI_selection_on_video(video_path):
    """
    Permite seleccionar puntos de interés (ROI) en un video.
    
    Parameters:
        video_path (str): Ruta del video de entrada.
    
    Returns:
        list: Lista de puntos ROI seleccionados.
    """
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: No se pudo abrir el video en {video_path}")
        return None

    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:  # Verificar si se pudo leer el fotograma
            break

        # Redimensionar el fotograma
        frame = downsample_image(frame, (800, 600))
        if frame is None:  # Verificar si la redimensión fue exitosa
            print("Error: No se pudo redimensionar el fotograma.")
            break

        frames.append(frame)

    cap.release()

    if not frames:
        print("Error: No se cargaron fotogramas del video.")
        return None

    # Seleccionar puntos ROI en las imágenes
    roi_points = select_points_on_images(frames)
    return roi_points

In [26]:
# Comprobar que la regio de interes es correcta
# Read the image
video = cv2.VideoCapture('data/test_video/TEST_1080.mov')
ret, frame = video.read()

# Seleccionar los puntos de la región de interés
roi_points = execute_ROI_selection_on_video('data/test_video/TEST_1080.mov')

Point 1 selected: (227, 535)
Point 2 selected: (385, 414)
Point 3 selected: (531, 403)
Point 4 selected: (778, 525)
4 points selected. Press 'q' to continue.
Point 1 selected: (167, 537)
Point 2 selected: (379, 407)
Point 3 selected: (537, 407)
Point 4 selected: (771, 499)
4 points selected. Press 'q' to continue.
Point 1 selected: (166, 551)
Point 2 selected: (372, 419)
Point 3 selected: (572, 420)
Point 4 selected: (793, 533)
4 points selected. Press 'q' to continue.
Point 1 selected: (220, 542)
Point 2 selected: (371, 410)
Point 3 selected: (530, 404)
Point 4 selected: (790, 518)
4 points selected. Press 'q' to continue.
Point 1 selected: (200, 537)
Point 2 selected: (395, 412)
Point 3 selected: (537, 408)
Point 4 selected: (780, 513)
4 points selected. Press 'q' to continue.
Point 1 selected: (185, 548)
Point 2 selected: (373, 423)
Point 3 selected: (541, 411)
Point 4 selected: (768, 518)
4 points selected. Press 'q' to continue.
[[194.16666667 541.66666667]
 [379.16666667 414.1666

In [42]:
# mostrar un video con la region de interes seleccionada
def downsample_video(input_path, output_path): 
    video = cv2.VideoCapture(input_path)
    if not video.isOpened():
        print(f"Error: No se pudo abrir el video en {input_path}")
        return
    
    # Crear un objeto VideoWriter para guardar el video
    frame_width = 800
    frame_height = 600
    fps = video.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break

        # Redimensionar el fotograma
        frame = downsample_image(frame, (800, 600))
        out.write(frame)

    video.release()
    out.release()

    print(f"Video redimensionado guardado en {output_path}")

time_ini = time.time()
input_path = 'data/test_video/TEST_1080.mov'
output_path = 'data/test_video/TEST_1080_downsampled.mov'

downsample_video(input_path, output_path)
time_end = time.time()
print(f"Tiempo de ejecución: {time_end - time_ini:.2f} segundos")


Video redimensionado guardado en data/test_video/TEST_1080_downsampled.mov
Tiempo de ejecución: 29.16 segundos


In [46]:

def process_ROI_downsample(input_downsampled_path, output_path, roi_points, downsampled_size=(800, 600)): 
    video = cv2.VideoCapture(input_downsampled_path)
    if not video.isOpened():
        print(f"Error: No se pudo abrir el video en {input_downsampled_path}")
        return
        
    # Create a VideoWriter object to save the video
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    if frame_width != downsampled_size[0] or frame_height != downsampled_size[1]:
        print("Error: El tamaño del video redimensionado no coincide con el tamaño de entrada.")
        return

    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break

        # Aplicar la transformación de perspectiva
        warped_frame, _, _ = apply_perspective_transform(frame, roi_points)
        
        out.write(warped_frame)
        # aply the gradient to the warped frame
        binary_warped = combined_gradient(warped_frame)
        


    video.release()
    out.release()

    print(f"Video procesado guardado en {output_path}")

time_ini = time.time()
input_downsampled_path = 'data/test_video/TEST_1080_downsampled.mp4'
output_path = 'data/test_video/TEST_1080_processed.mp4'

process_ROI_downsample(input_downsampled_path, output_path, roi_points)
time_end = time.time()
print(f"Tiempo de ejecución: {time_end - time_ini:.2f} segundos")


Video procesado guardado en data/test_video/TEST_1080_processed.mp4
Tiempo de ejecución: 97.97 segundos


In [73]:
buff = 0  # Mover 'buff' fuera de la función para que se mantenga entre fotogramas

def process_frame_downsampled(frame, roi_points):
    # Aplicar la transformación de perspectiva
    global buff
    warped_frame, dst_points, src_points = apply_perspective_transform(frame, roi_points)
    binary_warped = combined_gradient(warped_frame)
    
    histogram = calculate_histogram(binary_warped)
    left_base, right_base = find_lane_base(histogram)
    left_fit, right_fit, _, _, window_coords = perform_sliding_window(binary_warped, left_base, right_base)

    result = frame  # Valor predeterminado

    # Crear la imagen con las líneas de los carriles
    if left_fit is not None and right_fit is not None:
        ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])
        left_fitx = left_fit[0] * ploty**2 + left_fit[1] * ploty + left_fit[2]
        right_fitx = right_fit[0] * ploty**2 + right_fit[1] * ploty + right_fit[2]

        # Crear una imagen para dibujar las líneas de los carriles
        warp_zero = np.zeros_like(binary_warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recastear puntos para cv2.fillPoly
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Dibujar los carriles
        cv2.fillPoly(color_warp, np.int_([pts]), (0, 255, 0))
        Minv = cv2.getPerspectiveTransform(dst_points, src_points)
        newwarp = cv2.warpPerspective(color_warp, Minv, (frame.shape[1], frame.shape[0]))
        weighted_wrap = cv2.addWeighted(frame, 1, newwarp, 0.3, 0)

        # Agregar curvatura y dirección
        if buff == 30:  # Mostrar solo cada 30 fotogramas
            y_eval = np.max(ploty)
            left_curvature, right_curvature = calculate_curvature(left_fit, right_fit, y_eval)
            direction = detect_car_direction(left_fit, right_fit)
            curvature_text = f"Curvatura: {left_curvature:.2f} m, {right_curvature:.2f} m"
            direction_text = f"Dirección: {direction}"
            sbuff = 0  # Reiniciar el contador
        else:
            curvature_text = direction_text = ""

        buff += 1

        # Dibujar texto en el frame procesado
        cv2.putText(weighted_wrap, curvature_text, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        cv2.putText(weighted_wrap, direction_text, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        result = weighted_wrap

    return result


def process_video_downsampled(input_path, output_path, roi_points):
    video = cv2.VideoCapture(input_path)
    if not video.isOpened():
        print(f"Error: No se pudo abrir el video en {input_path}")
        return

    # Crear un objeto VideoWriter para guardar el video
    frame_width = int(video.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = video.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

    print(f"Información del video de entrada: {frame_width}x{frame_height}, {fps:.2f} FPS")

    while video.isOpened():
        ret, frame = video.read()
        if not ret:
            break

        # Procesar el fotograma
        processed_frame = process_frame_downsampled(frame, roi_points)
        out.write(processed_frame)

    video.release()
    out.release()

    print(f"Video procesado guardado en {output_path}")

time_ini = time.time()
input_path = 'data/test_video/TEST_1080_downsampled.mp4'
output_path = 'data/test_video/TEST_1080_processed.mp4'

process_video_downsampled(input_path, output_path, roi_points)
time_end = time.time()
print(f"Tiempo de ejecución: {time_end - time_ini:.2f} segundos")






    

Información del video de entrada: 800x600, 58.53 FPS
Video procesado guardado en data/test_video/TEST_1080_processed.mp4
Tiempo de ejecución: 203.92 segundos


: 